In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [96.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http:

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
!pip install sentence-transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 5.3 MB 20.9 MB/s 
     |████████████████████████████████| 1.3 MB 27.6 MB/s 
     |████████████████████████████████| 163 kB 75.7 MB/s 
     |████████████████████████████████| 7.6 MB 73.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=713a8ea6f52cd612aa23b8a64536d79a8bc01a2598bfbcb0a930212b73ec3c06
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from glob import glob
import re
import json
from tqdm import tqdm

import numpy as np
import pandas as pd
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch


In [6]:
def load_data(file, sheet_name):
    return pd.read_excel(file, sheet_name=sheet_name)

In [7]:
def load_all_data(path):
    files = glob(path+'/*.xlsx')
    df = pd.DataFrame()
    for file in files:
        df = df.append(load_data(file, '뉴스'))
    return df

In [8]:
def text_cleaning(x):
    mail_del = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z-.]+)", "", str(x))
    meta_del = re.sub("[\r\n\xa0]", "", str(mail_del))
    name_del = re.sub("(\.\s+[ㄱ-ㅎ가-힣]+\s[기]+[자]+)", "", str(meta_del))
    clean_text = re.sub("[^\w\s^.]", " ", name_del)
    
    return clean_text

In [19]:
def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    if candidate == None: return False
    return [words_vals[idx] for idx in candidate]

In [10]:
class mySBERT(SentenceTransformer):
    def __init__(self, path, modules=None):
        super().__init__(path, modules)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = self._first_module().tokenizer

    def encode(self, sentences, batch_size=32, show_progress_bar=None, output_value='token_embeddings', convert_to_numpy=True, convert_to_tensor=False, is_pretokenized=False):
        return super().encode(sentences, batch_size, show_progress_bar, output_value, convert_to_numpy, convert_to_tensor, is_pretokenized)

    def tokenize(self, sentences):
        return self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    
    def to(self, device):
        self.device = device
        return self

In [20]:
for i in range(1,5):
    path = "/content/drive/MyDrive/aida/dataset"
    save_path = "/content/drive/MyDrive/aida"
    if i==1:
        path += "/임대차3법(54,752건)"
        save_path += "/rent.json"
    if i==2:
        path += "/중대재해처벌법(40,056건)"
        save_path += "/accident.json"
    if i==3:
        path += "/차별금지법(59,421건)" 
        save_path += "/discrimination.json"
    if i==4:
        path += "/탄소중립(59,295건)" 
        save_path += "/neutral.json"

    print(path)
    print(save_path)

    data = load_all_data(path)
    data.columns = data.iloc[0]
    data.drop(0, inplace=True, axis=0)
    data = data.dropna(axis=0, how='any')

    data['내용'] = data['내용'].map(text_cleaning)

    okt = Okt()
    model = SentenceTransformer('jhgan/ko-sbert-multitask')

    keywords = []
    np.random.seed(42)
    #10000
    for i in tqdm(np.random.choice(data.index, 10000)):
        if len(data.loc[i]['내용']) == 0 or len(data.loc[i]['작성일']) == 0:
              continue
        tokenized_doc = okt.pos(data.iloc[i]['내용'], norm=True, stem=True)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

        n_gram_range = (1,2)

        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()

        candidate_embeddings = model.encode(candidates, convert_to_tensor=True)
        doc_embedding = model.encode([tokenized_nouns], convert_to_tensor=True)
        
        max_sum_simmilar = max_sum_sim(doc_embedding.cpu(), candidate_embeddings.cpu(), candidates, 5, 10)
        if max_sum_simmilar == False:
            continue
        keywords.append({data.iloc[i]['작성일']: max_sum_simmilar})

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(keywords, f, ensure_ascii=False, indent=4)

/content/drive/MyDrive/aida/dataset/중대재해처벌법(40,056건)
/content/drive/MyDrive/aida/accident.json


100%|██████████| 10000/10000 [51:23<00:00,  3.24it/s]


/content/drive/MyDrive/aida/dataset/차별금지법(59,421건)
/content/drive/MyDrive/aida/discrimination.json


100%|██████████| 10000/10000 [54:42<00:00,  3.05it/s]


/content/drive/MyDrive/aida/dataset/탄소중립(59,295건)
/content/drive/MyDrive/aida/neutral.json


100%|██████████| 10000/10000 [54:12<00:00,  3.07it/s]
